In [1]:
# 采用Inception 模块 比VGG 更深，但是却比VGG 的参数更少，因为其去掉了后面的全连接层，
#参数大幅度减少

In [2]:
# Inception
import torch
from torch import nn
import numpy as np
from torch.autograd import Variable
from torchvision.datasets import CIFAR10


In [3]:
# 定义一个卷积加一个rule 激活函数和一个batchnorm 作为一个基本的层结构

def conv_relu(in_channel,out_channel,kernel,stride=1,padding=0):
    layer=nn.Sequential(
        nn.Conv2d(in_channel,out_channel,kernel,stride,padding),
        nn.BatchNorm2d(out_channel,eps=1e-3)
    
    
    )
    return layer


In [9]:
class inception(nn.Module):
    def __init__(self,in_channel,out1_1,out2_1,out2_3,out3_1,out3_5,out4_1):
        super(inception,self).__init__()
        # 第一条线路
        self.branch1x1=conv_relu(in_channel,out1_1,1)
        # 第二条线路
        self.branch3x3=nn.Sequential(
            conv_relu(in_channel,out2_1,1),
            conv_relu(out2_1,out2_3,3,padding=1)
        )
        # 第三条路线
        self.branch5x5=nn.Sequential(
            conv_relu(in_channel,out3_1,1),
            conv_relu(out3_1,out3_5,5,padding=2)
        )
        # 第四条路线
        self.branch_pool=nn.Sequential(
            nn.MaxPool2d(3,stride=1,padding=1),
            conv_relu(in_channel,out4_1,1)
            
        )
    def forward( self,x):
        f1=self.branch1x1(x)
        f2=self.branch3x3(x)
        f3=self.branch5x5(x)
        f4=self.branch_pool(x)
        output=torch.cat((f1,f2,f3,f4),dim=1)
        return output
    

In [11]:
test_net=inception(3,	64,	48,	64,	64,	96,	32)
test_x=Variable(torch.zeros(1,3,96,96))
print(test_x.shape[1],test_x.shape[2],test_x.shape[3])
test_y=test_net(test_x)
print(test_y.shape[1],test_y.shape[2],test_y.shape[3])

3 96 96
256 96 96


In [12]:
# 大小没有变化。通道维度变多了，
# 下面定义了GoogLeNet ,GoogLeNet 可以看作是很多个Inception 模块的串联，# 原论文中
# 使用了多个输出来解决梯度消失的问题，我们这里只定义了一个简单版本的GoogLeNet ，
#简单为一个输出

In [16]:
class googlenet(nn.Module):
    def __init__(self,in_channel,num_classes,verbose=False):
        super(googlenet,self).__init__()
        self.verbose=verbose
        self.block1=nn.Sequential(
            conv_relu(in_channel,out_channel=64,kernel=7,stride=2,padding=3),
            nn.MaxPool2d(3,2)
        
        )
        self.block2=nn.Sequential(
            conv_relu(64,64,kernel=1),
            conv_relu(64,192,kernel=3,padding=1),
            nn.MaxPool2d(3,2)
        )
        
        self.block3=nn.Sequential(
            inception(192,64,96,128,16,32,32),
            inception(256,128,128,192,32,96,64),
            nn.MaxPool2d(3,2)
        )
        
        self.block4=nn.Sequential(
            inception(480,192,96,208,16,48,64),
            inception(512,160,112,224,24,64,64),
            inception(512,128,128,256,24,64,64),
            inception(512,112,144,288,32,64,64),
            inception(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2)
        
        )
        self.block5=nn.Sequential(
            inception(832,256,160,320,32,128,128),
            inception(832,384,182,384,48,128,128),
            nn.AvgPool2d(2)
        )
        self.classifier=nn.Linear(1024,num_classes)
        
    def forward(self,x):
            x=self.block1(x)
            if self.verbose:
                print(x.shape)
            x=self.block2(x)
            if self.verbose:
                print(x.shape)
            x=self.block3(x)
            if self.verbose:
                print(x.shape)  
                
            x=self.block4(x)
            if self.verbose:
                print(x.shape)
            x=self.block5(x)
            if self.verbose:
                print(x.shape)
            x=x.view(x.shape[0],-1)
            x=self.classifier(x)
            return x

In [17]:
test_net=googlenet(3,10,True)
test_x=Variable(torch.zeros(1,3,96,96))
test_y=test_net(test_x)
print(test_y.shape)

torch.Size([1, 64, 23, 23])
torch.Size([1, 192, 11, 11])
torch.Size([1, 480, 5, 5])
torch.Size([1, 832, 2, 2])
torch.Size([1, 1024, 1, 1])
torch.Size([1, 10])


In [ ]:
# 通道不断增加，尺寸不断减小